In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [77]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from src.features import build_features
from src.visualization import visualize

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

# 1. Descripción del problema a resolver

En este cuadernillo se quiere cuantificar la brecha sanitaria que existe para diversos problemas
de salud relevantes para el Instituto Nacional del Tórax. En específico, se quiere saber si
tales problemas son resueltos por el sistema de salud o no. Para esto, se contrastará la información de 3 fuentes de información:

1. Casos teóricos por problema de salud según incidencia (ET).
2. Casos reales de egresos hospitalarios a nivel país por problema de salud (EP).
3. Casos reales de egresos hospitalarios del INT por problema de salud (EI).

En primer lugar, se calculará la brecha de egresos hospitalarios teóricos y a nivel país. O sea:

$Brecha Pais= ET - EP$

Si $ET > EP$, entonces existe una brecha, y el sistema sanitario ha fallado en resolver el problema de salud. En caso contrario, si se ha resuelto el problema de salud.

En nuestro caso, se quieren identificar los problemas de salud donde $Brecha Pais > 0$, ya que el INT podría hacerse cargo de tales casos sin resolver en el nuevo hospital.

Una vez identificado los problemas de salud donde hay capacidad de crecimiento, se calculará el porcentaje de crecimiento que tendría el INT al asumir el 100% de la brecha del país. Para esto, se utilizará la siguiente fórmula:

$Casos Nuevos del INT = EI + Brecha Pais$

$PorcentajeDeCrecimientoINT = \frac{Casos Nuevos del INT - EI}{EI} * 100$

Esto nos permitirá identificar qué diagnósticos necesitarán un mayor porcentaje de crecimiento que otros.

## Ejemplo de cálculo

A modo de ejemplo, se mostrará un cálculo manual:

$ET = 1055; EP = 493; EI = 220$

Con estos datos, podemos calcular que:

$Brecha Pais = 1055 - 493 = 562$

Por lo tanto, faltan 562 casos que el sistema sanitario falla en atender. Ahora, asumiendo que el INT se hará cargo del 100% de estos casos (sólo para temas de simplicidad. Sin embargo, en ciertos casos es imposible, especialmente tomando en cuenta la cantidad de casos que recibe actualmente el recinto), es posible calcular el % de crecimiento para esta brecha.

$Casos Nuevos del INT = 220 + 562 = 782$
$PorcentajeDeCrecimientoINT = \frac{782 - 220}{220} \* 100 = 255\% $

Por lo tanto, el INT debería crecer en un 155% de su capacidad actual para suplir el 100% de la necesidad sanitaria en el diagnóstico X.


## Extracción de egresos teóricos (ET)


In [31]:
# Carga la poblacion teorica de los 45 diagnosticos mas relevantes para el INT
poblacion_teorica = pd.read_excel("../data/processed/proyeccion_problemas_de_salud.xlsx")


# Solamente deja el codigo CIE de los problemas de salud

poblacion_teorica["Diagnostico"] = poblacion_teorica.Diagnostico.str.split().str[0]

# Deja los egresos teoricos a nivel pais

poblacion_teorica = poblacion_teorica.query("Estrato == 'Pais'")


# Extrae los codigos de los diagnosticos relevantes y los deja como indice

DIAGS_RELEVANTES = poblacion_teorica.Diagnostico.unique()

poblacion_teorica = poblacion_teorica.set_index("Diagnostico").sort_index()


# Selecciona los egresos teoricos hasta 2020

poblacion_teorica = poblacion_teorica[[i for i in range(2017, 2021)]]

# De los casos totales, se asume que una parte (0.8) se resuelve en hospitalizacion
RATIO_HOSPITALIZADOS = 0.8
RATIO_AMBULATORIO = 1 - RATIO_HOSPITALIZADOS

# Calcula la cantidad de casos en hospitalizados y ambulatorio. Ademas, redondea los casos
poblacion_teorica_hospitalizados = round(poblacion_teorica * RATIO_HOSPITALIZADOS)
poblacion_teorica_ambulatorio = round(poblacion_teorica * RATIO_AMBULATORIO)

## Extracción de egresos a nivel país (EP)


In [20]:
# Lee egresos hospitalarios a nivel país
egresos_pais = pd.read_csv(

    "../data/processed/ranking_nacional_egresos.csv",

    sep=";",

    encoding="latin-1",

    usecols=[

        "ANO_EGRESO",

        "ESTABLECIMIENTO_SALUD",

        "DIAG1",
        "n_egresos",
        "dias_estada_totales",

        "n_int_q",

        "n_muertos",

    ],

)


# Deja solamente los 45 egresos relevantes para el INT y desde 2017

egresos_pais = egresos_pais[egresos_pais["DIAG1"].isin(DIAGS_RELEVANTES)]

egresos_pais = egresos_pais.query("ANO_EGRESO >= 2017")

## Extracción de egresos INT (EI)


In [21]:
# Lee egresos hospitalarios de INT
egresos_int = pd.read_csv("../data/processed/egresos_torax_mes_y_dia.csv")

# Deja solamente los 45 egresos relevantes para el INT y desde 2017

egresos_int = egresos_int[egresos_int["DIAG1"].isin(DIAGS_RELEVANTES)]

egresos_int = egresos_int.query("ANO_EGRESO >= 2017")

In [35]:
# Transforma la tabla de egresos del pais a formato de analisis
tabla_dinamica_egresos_pais = pd.pivot_table(
    egresos_pais,

    index="DIAG1",

    columns="ANO_EGRESO",

    values=["n_egresos"],
    aggfunc="sum",

    fill_value=0,

).sort_index()


# Transforma la tabla de egresos int a formato de analisis

tabla_dinamica_egresos_int = pd.pivot_table(
    egresos_int,

    index="DIAG1",

    columns="ANO_EGRESO",

    values=["n_egresos"],
    aggfunc="sum",

    fill_value=0,

).sort_index()

tabla_dinamica_egresos_pais.columns = tabla_dinamica_egresos_pais.columns.droplevel(0)
tabla_dinamica_egresos_int.columns = tabla_dinamica_egresos_int.columns.droplevel(0)

Una vez extraído los datos y haberlos transformados para realizar su análisis, se procederá a calcular las respectivas brechas.

En primer lugar, se obtendrá la brecha a nivel país por problema de salud. Recordemos que la fórmula de cálculo es:

$Brecha Pais= ET - EP$


In [79]:
brecha_pais = (poblacion_teorica_hospitalizados - tabla_dinamica_egresos_pais).astype("Int32")
brecha_pais.style.map(visualize.color_negative_red)

,2017,2018,2019,2020
Diagnostico,,,,
C33X,-10,-3,-2,2
C340,854,892,907,971
C341,645,557,562,758
C342,967,967,1003,1026
C343,759,742,776,851
C381,136,139,131,145
C384,-34,-39,-47,-38
C450,-76,-71,-70,-48
C780,644,656,625,748


Los resultados indican que en gran parte de los diagnósticos faltó suplir la demanada del problema
de salud (ej: C340, C341, C342, C343, etc). Los resultados se pueden dividir en distintos tipos
de
problema de salud:

1. **Cáncer de Pulmón (C33X a C782)**: En todos los diagnósticos faltó suplir la necesidad del
   país (exceptuando C33X, C384, C450). EL diagnóstico con una mayor necesidad de atención fue
   el C342, con una falta de 1026 egresos.

2. **Patologías Valvulares (I051 a I712)**: En todos los diagnósticos faltó suplir la necesidad del
   país (exceptuando I420).


Con estos resultados, es posible identificar todos los diagnósticos donde es necesario cerrar la brecha de atención de salud. Se seleccionaran los diagnósticos donde en al menos 1 de los años entre 2017 y 2020 exista una $Brecha País > 0.$


In [98]:
# Selecciona solamente los diagnosticos donde exista una brecha > 0 en alguno de los anios
brecha_pais_a_resolver = brecha_pais[(brecha_pais > 0).any(axis=1)]
# Obtiene los codigos de los diagnosticos con una brecha
diagnosticos_con_necesidad_sanitaria = brecha_pais_a_resolver.index

Una vez identificado los diagnosticos con necesidad sanitaria, es posible calcular cuánto debería crecer el INT en cada uno de estos diagnósticos para suplir tal brecha.


In [147]:
# Calcula los nuevos casos que deberia atender, asumiendo que se hara cargo del 100% de la brecha
casos_nuevos_a_atender_int = brecha_pais + tabla_dinamica_egresos_int

# Calcula el porcentaje de crecimiento, tomando en cuenta los casos actuales del INT
porcentaje_crecimiento_int = (
    (casos_nuevos_a_atender_int - tabla_dinamica_egresos_int) / tabla_dinamica_egresos_int
) * 100
# Redondea el porcentaje de crecimiento
porcentaje_crecimiento_int = porcentaje_crecimiento_int.round().astype("Int32")

In [148]:
porcentaje_crecimiento_int_diags_a_atender = porcentaje_crecimiento_int.loc[
    diagnosticos_con_necesidad_sanitaria
]

In [150]:
porcentaje_crecimiento_int_diags_a_atender

,2017,2018,2019,2020
Diagnostico,,,,
C33X,-1000,-300,-50,29
C340,4067,5247,3779,13871
C341,358,253,255,997
C342,5089,4029,5015,14657
C343,643,598,647,2239
C381,907,662,689,2900
C780,1130,1041,1008,4987
C782,3000,1598,2164,1714
D143,2622,2070,3196,16017


## Cálculo de procedimientos teniendo en cuenta la brecha a atender en el INT


In [143]:
procedimientos = pd.read_excel("../data/processed/canasta_procedimientos_hospitalizados.xlsx")
procedimientos["DIAGNOSTICO1"] = (
    procedimientos["DIAGNOSTICO1"].str.replace(".", "").str.ljust(4, "X")
)

In [158]:
procedimientos_con_crecimiento = procedimientos.set_index("DIAGNOSTICO1").merge(
    ratio_crecimiento_promedio, how="inner", left_index=True, right_index=True
)

In [159]:
procedimientos_con_crecimiento["cantidad_procedimientos_aumentados"] = (
    procedimientos_con_crecimiento["cantidad_procedimientos"]
    * procedimientos_con_crecimiento["indice_crecimiento"]
)

procedimientos_con_crecimiento["cantidad_pacientes_distintos_aumentados"] = (
    procedimientos_con_crecimiento["cantidad_pacientes_distintos"]
    * procedimientos_con_crecimiento["indice_crecimiento"]
)

In [160]:
COLUMNAS_UTILES = [
    "ANIO_EGRESO",
    "procedimiento",
    "Descripción_x",
    "Descripción_y",
    "cantidad_procedimientos",
    "cantidad_pacientes_distintos",
    "cantidad_procedimientos_aumentados",
    "cantidad_pacientes_distintos_aumentados",
    "indice_crecimiento",
]
procedimientos_con_crecimiento = procedimientos_con_crecimiento[COLUMNAS_UTILES]

In [161]:
display(procedimientos_con_crecimiento)

,ANIO_EGRESO,procedimiento,Descripción_x,Descripción_y,cantidad_procedimientos,cantidad_pacientes_distintos,cantidad_procedimientos_aumentados,cantidad_pacientes_distintos_aumentados,indice_crecimiento
DIAGNOSTICO1,,,,,,,,,
C33X,2019,99.29,INYECCION SUSTANCIA TERAPEUTICA O PROFILACTICA...,Tumor maligno de la tráquea,4,4,4.0284,4.0284,1.0071
C33X,2019,99.23,INYECCION ESTEROIDE,Tumor maligno de la tráquea,4,4,4.0284,4.0284,1.0071
C33X,2019,99.21,INYECCION ANTIBIOTICO,Tumor maligno de la tráquea,4,4,4.0284,4.0284,1.0071
C33X,2019,93.96,ENRIQUECIMIENTO OXIGENO OTRO,Tumor maligno de la tráquea,4,4,4.0284,4.0284,1.0071
C33X,2019,93.94,AEROSOLTERAPIA,Tumor maligno de la tráquea,4,4,4.0284,4.0284,1.0071
...,...,...,...,...,...,...,...,...,...
Z450,2021,0.23,IMAGEN INTRAVASCULAR VASO PERIFERICO,Asistencia y ajuste de marcapaso cardíaco,1,107,NaN,NaN,NaN
Z450,2021,88.41,ARTERIOGRAFIA ARTERIA CEREBRAL,Asistencia y ajuste de marcapaso cardíaco,1,107,NaN,NaN,NaN
Z450,2021,88.21,RADIOGRAFIA HUESO HOMBRO Y BRAZO,Asistencia y ajuste de marcapaso cardíaco,1,107,NaN,NaN,NaN


In [162]:
procedimientos_con_crecimiento.to_excel("../data/interim/procedimientos_aumentados.xlsx")